# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit.ephemeris import Ephemeris, EphemerisPostProcessor

In [2]:
eph1 = Ephemeris(ssos="default solar system")
t = Time("2024-12-01 00:00")

eph1.state(t)["ceres"]

{'x': <Quantity [ 1.95941058, -1.83557751, -1.26554065] AU>,
 'v': <Quantity [0.00718976, 0.00621272, 0.00146799] AU / d>,
 'a': <Quantity [0., 0., 0.] AU / d2>,
 'log_gm': Array(-29.59967161, dtype=float64, weak_type=True)}

In [3]:
obj = Horizons(id="ceres", location="@0", epochs=t.tdb.jd)
vecs = obj.vectors(refplane="earth")
vecs

targetname,datetime_jd,datetime_str,H,G,x,y,z,vx,vy,vz,lighttime,range,range_rate
---,d,---,mag,---,AU,AU,AU,AU / d,AU / d,AU / d,d,AU,AU / d
str17,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1 Ceres (A801 AA),2460645.50080073,A.D. 2024-Dec-01 00:01:09.1831,3.34,0.12,1.959410581544375,-1.835577502420488,-1.265540654845034,0.007189759851123859,0.006212717988504686,0.001467994779189476,0.01714290956762958,2.968202780054944,0.0002782690905952506


In [4]:
err = (
    eph1.state(t)["ceres"]["x"]
    - jnp.array([vecs["x"][0], vecs["y"][0], vecs["z"][0]]) * u.au
)
print(err)
print(jnp.linalg.norm(err) * u.au.to(u.m))

[-1.55797930e-09 -2.80592927e-09  1.10484955e-11] AU
480.1290143863208


In [5]:
# looks like we're getting sub-meter agreement with the planets,
# but a few km off with the asteroids

from jorbit.data.constants import ALL_PLANET_IDS, LARGE_ASTEROID_IDS

components = list(eph1.state(t).keys())

horizons_positions = []
for component in components:
    if component in ALL_PLANET_IDS:
        component = ALL_PLANET_IDS[component]
        obj = Horizons(id=component, location="@0", epochs=t.tdb.jd)
    elif component in LARGE_ASTEROID_IDS:
        obj = Horizons(
            id=component, location="@0", epochs=t.tdb.jd, id_type="smallbody"
        )

    vecs = obj.vectors(refplane="earth")
    horizons_positions.append(
        jnp.array([vecs["x"][0], vecs["y"][0], vecs["z"][0]]) * u.au
    )

for component, pos in zip(components, horizons_positions):
    err = eph1.state(t)[component]["x"] - pos
    print(component, ":", jnp.linalg.norm(err) * u.au.to(u.m))

sun : 0.00015719823206890104
mercury : 0.6437843598026342
venus : 0.3906539067908675
earth : 0.33822920590193944
mars : 0.26049494997121736
jupiter : 0.14997553511703793
saturn : 0.10694395483529756
uranus : 0.07374597980523129
neptune : 0.0612630520224855
pluto : 0.05948107674801526
ceres : 480.1290143863208
pallas : 97365.1312281726
juno : 158224.1537592059
vesta : 12904.228069566292
iris : 114288.13017226166
hygiea : 27019.768060730596
eunomia : 44801.44092217856
psyche : 14382.70225222105
euphrosyne : 70394.44208465585
europa : 34258.77244040259
cybele : 132706.01947245933
sylvia : 40117.50615430843
thisbe : 19362.777854667354
camilla : 40254.63022917626
davida : 45575.83554590772
interamnia : 26066.69991732719


In [6]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit.utils.states import SystemState
from jorbit.data.constants import INV_SPEED_OF_LIGHT
from jorbit.integrators import ias15_evolve, initialize_ias15_integrator_state

t = Time("2024-12-01 00:00")

In [7]:
from jorbit.particle import Particle
from jorbit.system import System

obj = Horizons(id="274301", location="@0", epochs=t.tdb.jd)
vecs = obj.vectors(refplane="earth")

x = jnp.array([vecs["x"][0], vecs["y"][0], vecs["z"][0]])
v = jnp.array([vecs["vx"][0], vecs["vy"][0], vecs["vz"][0]])

p = Particle(x=x, v=v, time=t, name="wiki")
system = System(particles=[p], acceleration_func="newtonian planets")

print(system.acceleration_func(system.state))

system

[[ 2.61030935e-05 -2.90119924e-05 -8.52860259e-06]]


*************
jorbit System
 time: 2460645.50080073
 particles: [Particle: wiki]
*************

In [12]:
from jorbit.observation import Observations

obj = Horizons(id="274301", location="695", epochs=t.jd)
eph = obj.ephemerides(extra_precision=True, quantities="1")
coord = SkyCoord(eph["RA"][0], eph["DEC"][0], unit=(u.deg, u.deg), frame="icrs")
print(coord)

obs = Observations(
    observed_coordinates=coord,
    times=t,
    observatories="695@399",
    astrometric_uncertainties=1 * u.arcsec,
    verbose=True,
)
obs

<SkyCoord (ICRS): (ra, dec) in deg
    (152.01007338, 5.09857717)>


Observations with 1 set(s) of observations

In [40]:
@jax.jit
def on_sky(
    state: SystemState,
    acc_func,
    observer_position,
):
    a0 = acc_func(state)
    initial_integrator_state = initialize_ias15_integrator_state(a0)

    def scan_func(carry, scan_over):
        xz = carry
        earth_distance = jnp.linalg.norm(xz - observer_position)
        light_travel_time = earth_distance * INV_SPEED_OF_LIGHT

        positions, final_system_state, final_integrator_state = ias15_evolve(
            state,
            acc_func,
            jnp.array([state.time - light_travel_time]),
            initial_integrator_state,
            n_steps=5,
        )

        return final_system_state.positions, None

    xz = jax.lax.scan(scan_func, state.positions, None, length=2)[0]

    X = xz - observer_position
    calc_ra = jnp.mod(jnp.arctan2(X[:, 1], X[:, 0]) + 2 * jnp.pi, 2 * jnp.pi)
    calc_dec = jnp.pi / 2 - jnp.arccos(X[:, -1] / jnp.linalg.norm(X, axis=1))
    return calc_ra, calc_dec


a, b = on_sky(system.state, system.acceleration_func, obs.observer_positions[0])
c = SkyCoord(a, b, unit=(u.rad, u.rad))

c.separation(coord).to(u.microarcsecond)

<Angle [1.1507072] uarcsec>

In [41]:
%%timeit

on_sky(system.state, system.acceleration_func, obs.observer_positions[0])[
    0
].block_until_ready()

558 μs ± 1.46 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [39]:
@jax.jit
def tmp(state):
    a0 = system.acceleration_func(state)
    initial_integrator_state = initialize_ias15_integrator_state(a0)
    return initial_integrator_state


%timeit tmp(system.state).g.p0.block_until_ready()

170 μs ± 2.89 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
